In [7]:
#IMPORT LIBRARIES
from sklearn.neural_network import MLPRegressor
import numpy as np
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.datasets import make_blobs, make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import hvplot.pandas
import holoviews as hv
from prophet import Prophet
import datetime as dt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

%matplotlib inline

In [8]:
#READ CSV FILES
GMA_data = pd.read_csv("data/GMA_data.csv")
Houston_data = pd.read_csv("data/Houston_data.csv")

In [9]:
#Display GMA_data
display(GMA_data.head())

,Geo Market Area,ASP 12/2021 - 12/2022,ASP 12/2022 - 12/2023,% Change,Transactions 12/2021 - 12/2022,Transactions 12/2022 - 12/2023,% Change.1,MOI 11/2022,MOI 11/2023,% Change.2,Avg DOM 12/2021 - 12/2022,Avg DOM 12/2022 - 12/2023,% Change.3
0,1960/Cypress,"$305,022","$319,320",4.70%,290,205.0,-29.30%,1.5,2.6,72.10%,15.0,35.7,138.00%
1,1960/Cypress Creek North,"$321,283","$331,624",3.20%,411,245.0,-40.40%,2.2,4.6,107.50%,27.6,39.2,42.00%
2,1960/Cypress Creek South,"$265,879","$258,609",-2.70%,786,613.0,-22.00%,2.2,2.9,28.20%,23.3,40.2,72.50%
3,Aldine Area,"$237,692","$237,042",-0.30%,880,755.0,-14.20%,2.5,2.9,14.60%,23.2,38.1,64.20%
4,Alief,"$260,950","$262,756",0.70%,523,443.0,-15.30%,2.5,3.2,25.50%,21.8,31.6,45.00%


In [10]:
#Read Historic_data.csv
Historic_data = pd.read_csv("data/Historic_pricing.csv",
    index_col= 'Year',
    parse_dates=True,
    infer_datetime_format=True,
  skiprows=(1)
                )


#Display Historic_data
display(Historic_data.head())
display(Historic_data.tail())

C:\Users\vicky\AppData\Local\Temp\ipykernel_6880\3219733023.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  Historic_data = pd.read_csv("data/Historic_pricing.csv",


,Month,Listings*,Price,Price.1,Sqft,Listings,Listings.1,Listings.2,DOM,Orig. List Price
Year,,,,,,,,,,
2021-01-01,Jan,"4,815","$321,238","$255,000",$132,"7,202","9,088","6,097",43,96.70%
2021-01-01,Feb,"4,671","$348,928","$270,000",$139,"6,148","7,781","5,721",42,97.20%
2021-01-01,Mar,"7,327","$363,397","$285,000",$143,"8,831","7,184","7,889",39,97.80%
2021-01-01,Apr,"7,459","$372,590","$290,500",$148,"10,164","7,725","8,195",33,98.80%
2021-01-01,May,"7,856","$389,681","$302,500",$152,"10,280","7,938","8,341",28,99.60%


,Month,Listings*,Price,Price.1,Sqft,Listings,Listings.1,Listings.2,DOM,Orig. List Price
Year,,,,,,,,,,
2023-01-01,Apr,"5,244","$414,654","$320,000",$174,"8,265","12,023","6,027",43,96.30%
2023-01-01,May,"6,365","$430,551","$333,000",$177,"9,325","12,484","6,233",38,96.70%
2023-01-01,Jun,"6,503","$433,712","$340,000",$177,"10,373","14,191","5,845",34,96.70%
2023-01-01,Jul,"5,750","$424,059","$335,600",$175,"9,597","14,958","5,980",32,96.60%
2023-01-01,Aug,"6,207","$418,321","$331,500",$176,"9,284","15,740","5,677",34,96.10%


In [11]:
# Reset the index of the DataFrame
prophet_df = Historic_data.reset_index()


#Display Historic_data
display(Historic_data.head())

,Month,Listings*,Price,Price.1,Sqft,Listings,Listings.1,Listings.2,DOM,Orig. List Price
Year,,,,,,,,,,
2021-01-01,Jan,"4,815","$321,238","$255,000",$132,"7,202","9,088","6,097",43,96.70%
2021-01-01,Feb,"4,671","$348,928","$270,000",$139,"6,148","7,781","5,721",42,97.20%
2021-01-01,Mar,"7,327","$363,397","$285,000",$143,"8,831","7,184","7,889",39,97.80%
2021-01-01,Apr,"7,459","$372,590","$290,500",$148,"10,164","7,725","8,195",33,98.80%
2021-01-01,May,"7,856","$389,681","$302,500",$152,"10,280","7,938","8,341",28,99.60%


In [12]:
# Print d-types
print(Historic_data.dtypes)

Month               object
Listings*           object
Price               object
Price.1             object
Sqft                object
Listings            object
Listings.1          object
Listings.2          object
DOM                  int64
Orig. List Price    object
dtype: object


In [13]:
# Prepare the training data to be read into a prophet model
prophet_df.drop(columns=['Listings*','Price','Price.1','Sqft', 'Listings', 'Listings.1', 'Listings.2', 'Orig. List Price', 'Month', ], inplace=True)


# Rename the columns to names that Prophet recognizes
prophet_df.columns = ['ds', 'y']
prophet_df.head(15)

,ds,y
0,2021-01-01,43
1,2021-01-01,42
2,2021-01-01,39
3,2021-01-01,33
4,2021-01-01,28
5,2021-01-01,24
6,2021-01-01,21
7,2021-01-01,22
8,2021-01-01,24
9,2021-01-01,27


In [14]:

# Call the Prophet function and store as an object
m = Prophet()
m


In [15]:
# Fit the time series Prophet model
m.fit(prophet_df)



19:21:21 - cmdstanpy - INFO - Chain [1] start processing
19:21:24 - cmdstanpy - INFO - Chain [1] done processing


In [16]:
# Create a future DataFrame to hold predictions
# Make the prediction go out as far as (30 days)
future = m.make_future_dataframe(periods=720, freq='H')

# Review the DataFrame
display(future.head())
display(future.tail(10))


,ds
0,2021-01-01 00:00:00
1,2022-01-01 00:00:00
2,2023-01-01 00:00:00
3,2023-01-01 01:00:00
4,2023-01-01 02:00:00


,ds
713,2023-01-30 15:00:00
714,2023-01-30 16:00:00
715,2023-01-30 17:00:00
716,2023-01-30 18:00:00
717,2023-01-30 19:00:00
718,2023-01-30 20:00:00
719,2023-01-30 21:00:00
720,2023-01-30 22:00:00
721,2023-01-30 23:00:00
722,2023-01-31 00:00:00


In [17]:
# Make a forecast based on the future DataFrame
forecast = m.predict(future)

# Review the first five rows of the forecast DataFrame
display(forecast.head())
display(forecast.tail())

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2021-01-01 00:00:00,23.409411,19.796938,37.847650,23.409411,23.409411,5.257109,5.257109,5.257109,5.257109,5.257109,5.257109,0.0,0.0,0.0,28.666520
1,2022-01-01 00:00:00,26.201223,22.720523,41.998175,26.201223,26.201223,6.190739,6.190739,6.190739,6.190739,6.190739,6.190739,0.0,0.0,0.0,32.391963
2,2023-01-01 00:00:00,29.035525,27.903747,46.235084,29.035525,29.035525,7.381441,7.381441,7.381441,7.381441,7.381441,7.381441,0.0,0.0,0.0,36.416966
3,2023-01-01 01:00:00,29.035848,26.371344,45.610841,29.035848,29.035848,7.165197,7.165197,7.165197,7.165197,7.165197,7.165197,0.0,0.0,0.0,36.201046
4,2023-01-01 02:00:00,29.036172,27.126955,44.715475,29.036172,29.036172,6.956061,6.956061,6.956061,6.956061,6.956061,6.956061,0.0,0.0,0.0,35.992233


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
718,2023-01-30 20:00:00,29.267187,78.034946,95.922961,29.263989,29.271124,57.343852,57.343852,57.343852,57.343852,57.343852,57.343852,0.0,0.0,0.0,86.611040
719,2023-01-30 21:00:00,29.267511,76.499637,94.686436,29.264305,29.271469,56.025986,56.025986,56.025986,56.025986,56.025986,56.025986,0.0,0.0,0.0,85.293497
720,2023-01-30 22:00:00,29.267834,75.158553,93.052066,29.264622,29.271814,54.708076,54.708076,54.708076,54.708076,54.708076,54.708076,0.0,0.0,0.0,83.975910
721,2023-01-30 23:00:00,29.268158,73.668143,91.896329,29.264940,29.272162,53.390172,53.390172,53.390172,53.390172,53.390172,53.390172,0.0,0.0,0.0,82.658330
722,2023-01-31 00:00:00,29.268481,72.416457,90.561918,29.265257,29.272507,52.072323,52.072323,52.072323,52.072323,52.072323,52.072323,0.0,0.0,0.0,81.340805


In [19]:
# Define X
X = forecast.copy()
X.drop("ds", axis=1, inplace=True)
X.head()

,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,23.409411,19.796938,37.847650,23.409411,23.409411,5.257109,5.257109,5.257109,5.257109,5.257109,5.257109,0.0,0.0,0.0,28.666520
1,26.201223,22.720523,41.998175,26.201223,26.201223,6.190739,6.190739,6.190739,6.190739,6.190739,6.190739,0.0,0.0,0.0,32.391963
2,29.035525,27.903747,46.235084,29.035525,29.035525,7.381441,7.381441,7.381441,7.381441,7.381441,7.381441,0.0,0.0,0.0,36.416966
3,29.035848,26.371344,45.610841,29.035848,29.035848,7.165197,7.165197,7.165197,7.165197,7.165197,7.165197,0.0,0.0,0.0,36.201046
4,29.036172,27.126955,44.715475,29.036172,29.036172,6.956061,6.956061,6.956061,6.956061,6.956061,6.956061,0.0,0.0,0.0,35.992233


In [20]:
# Define target vector
y = forecast["ds"].values.reshape(-1, 1)
y[:5]

array([['2021-01-01T00:00:00.000000000'],
       ['2022-01-01T00:00:00.000000000'],
       ['2023-01-01T00:00:00.000000000'],
       ['2023-01-01T01:00:00.000000000'],
       ['2023-01-01T02:00:00.000000000']], dtype='datetime64[ns]')

In [21]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [22]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [23]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [24]:
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [25]:
# Create the random forest classifier instance
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [26]:
# Fit the model
rf_model = rf_model.fit(X_train_scaled, y_train)


C:\Users\vicky\Downloads\An\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [27]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

predictions

array(['2023-01-01T19:00:00.000000000', '2023-01-08T15:00:00.000000000',
       '2023-01-21T03:00:00.000000000', '2023-01-06T02:00:00.000000000',
       '2023-01-23T13:00:00.000000000', '2023-01-22T19:00:00.000000000',
       '2023-01-03T16:00:00.000000000', '2023-01-03T15:00:00.000000000',
       '2023-01-03T03:00:00.000000000', '2023-01-21T08:00:00.000000000',
       '2023-01-30T14:00:00.000000000', '2023-01-16T13:00:00.000000000',
       '2023-01-06T18:00:00.000000000', '2023-01-30T23:00:00.000000000',
       '2023-01-11T01:00:00.000000000', '2023-01-08T01:00:00.000000000',
       '2023-01-05T15:00:00.000000000', '2023-01-10T04:00:00.000000000',
       '2023-01-11T15:00:00.000000000', '2023-01-21T13:00:00.000000000',
       '2023-01-20T07:00:00.000000000', '2023-01-29T08:00:00.000000000',
       '2023-01-18T04:00:00.000000000', '2023-01-25T00:00:00.000000000',
       '2023-01-23T04:00:00.000000000', '2023-01-27T11:00:00.000000000',
       '2023-01-03T12:00:00.000000000', '2023-01-07

In [28]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

ValueError: Shape of passed values is (340, 340), indices imply (2, 2)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [29]:
# Display the average forecasted DOM for FEB 2023
forecast.mean()

ds                            2023-01-14 10:39:20.165975296
trend                                             29.139979
yhat_lower                                       170.247548
yhat_upper                                       188.542245
trend_lower                                       29.138978
trend_upper                                       29.141195
additive_terms                                   150.255065
additive_terms_lower                             150.255065
additive_terms_upper                             150.255065
yearly                                           150.255065
yearly_lower                                     150.255065
yearly_upper                                     150.255065
multiplicative_terms                                    0.0
multiplicative_terms_lower                              0.0
multiplicative_terms_upper                              0.0
yhat                                             179.395044
dtype: object